In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

#optional setting 
#pd.set_option('display.max_rows', 150)

In [2]:
season_episode_list =[]

for season in range(1,4):
    if season != 3:
        for episode in range(1,21):
            ep_num = str(episode)
            if len(ep_num) <= 1:
                ep_num = str(0)+ep_num
            else: 
                ep_num
            se_ep = str(season) + ep_num
            season_episode_list.append(se_ep)
    else:
        for episode in range(1,22):
            ep_num = str(episode)
            if len(ep_num) <= 1:
                ep_num = str(0)+ep_num
            else: 
                ep_num
            se_ep = str(season) + ep_num
            season_episode_list.append(se_ep)

In [ ]:
season_episode_list

In [ ]:
def getTitle(x):
    url_base = 'http://atla.avatarspirit.net/transcripts.php?num='
    season_episode_num = str(x) # in a function make this x
    response = requests.get(url_base + season_episode_num)
    
    #
    soup = BeautifulSoup(response.content, 'lxml')
    content = soup.find(class_='content')
    title = content.find(class_='welcome')
    title = title.get_text()
    title = title.split("-")[-1].strip()
    #print(title)
    return(title)

In [ ]:
df = pd.DataFrame(columns = ['SeasonEpisodeCode','Title'])

In [ ]:
#populate a df with SeasonEpisodeCodes & Titles
title_list = []
for x in season_episode_list:
    title_list.append(getTitle(x))

In [ ]:
#creation of df frame
df['SeasonEpisodeCode'] = season_episode_list
df['Title'] = title_list

df['Season'] = df['SeasonEpisodeCode']
df['EpisodeNum'] = df['SeasonEpisodeCode']

#obtain season#
def seasonNum(x):
    return(int(str(x)[0]))
#apply SeasonNum to df
df['Season'] = df['Season'].apply(seasonNum)

#obtain episode #
def episodeNum(x):
    return(int(str(x)[-2:]))
#apply EpisodeNum to df
df['EpisodeNum'] = df['EpisodeNum'].apply(episodeNum)

In [ ]:
df

In [3]:
#get episode text for cleaning
def RawScriptPull(x):
    url_base = 'http://atla.avatarspirit.net/transcripts.php?num='
    season_episode_num = str(x) # in a function make this x
    response = requests.get(url_base + season_episode_num)
    
    soup = BeautifulSoup(response.content, 'lxml')
    content = soup.find(class_='content')
    raw_script = content.find(class_='content')
    
    #extracts all text in italics - which are stage directions
    for linebreak in raw_script.find_all('i'):
        linebreak.extract()
    
    #extracts all for hyperlinks - which we don't need
    for linebreak in raw_script.find_all('a'):
        linebreak.extract()
    
    #extracts all the breaks
    for linebreak in raw_script.find_all('br'):
        linebreak.extract()
    #leaves us only the bolds... which we can seperate items on
    return(str(raw_script))
    
    #file = open(str(x) + ".txt","w")
    #file.write(str(script))

In [ ]:
#let's try doing this over two different functions

def scriptClean1(x):
    episode_code = locals()
    
    file_clean = re.sub(r'\([^()]*\)', '', RawScriptPull(x)) #references RawScriptPull so it can get cleaned 
    
    #removes data after the Ending Credits which is extra
    script = file_clean.split('[End Credits]')[0]

    #removes the data before Act I, which is also extra
    #unfortunetly, website uses 2 different conventions, so I split depending on which one it uses
    
    if "Act I</b></u>" in script:
        script_clean = script.split("Act I</b></u>")[1]
        #bold_split = script_clean.split('<b>')
    elif "Act I</u></b>" in script:
        script_clean = script.split("Act I</u></b>")[1]
        #bold_split = script_clean.split('<b>')
    return(script_clean)
    #bold_split = script_clean.split('<b>')   
    #return(bold_split)

In [ ]:
def boldSplit(x):
    script_clean = scriptClean1(x)
    bold_split = script_clean.split('<b>') 
    return(bold_split)

In [ ]:
#SO all of these work.... one you enter them one after the other... 

#scriptClean1(104)

#boldSplit(104)

#scriptClean2(104)

#dfConsolidateScript(101)

In [ ]:
#creates a list of lists with both character + relevant dialogue

def scriptClean2(x):
    episode_code = locals()
    bold_split = boldSplit(x)
    master_list = []
    for x in range(len(bold_split)):
        output = bold_split[x].split(':')
        master_list.append(output)
    
    #collects all the characters who speak
    character_list = []

    for x in range(len(master_list)):
        holder = master_list[x]
        if len(holder)>1:
            test = holder[0].strip()
            character_list.append(test.strip('</b>')) # remove the bold hyperlink to cleanup the data
    #collects what all the characters say
    dialogue_list = []

    for x in range(len(master_list)):
        holder = master_list[x]
        if len(holder)>1:
            test = holder[0].strip()
            dialogue_list.append(holder[1].strip('</b>')) # remove the bold hyperlink to cleanup the data 
    
    
    #create a list of episode numbers to add to df (feels inefficient and should be a better way to do this)
    episode_list = []
    
    for x in range(len(character_list)):
        episode_list.append(int(episode_code['x']))
    
    dfScript = pd.DataFrame()
    
    dfScript['episode_code'] = episode_list
    
    dfScript['character_list'] = character_list
    dfScript['dialogue_list'] = dialogue_list
    
    count = dfScript['dialogue_list'].str.split().str.len()
    count = list(count)
    dfScript['word_count']=count

    return(dfScript)

In [4]:
#creating a function to clean the script
#then puts it into a data frame
#and counts the number of words each character says

def scriptClean(x):
    episode_code = locals()
    
    file_clean = re.sub(r'\([^()]*\)', '', RawScriptPull(x)) #references RawScriptPull so it can get cleaned 
    
    #removes data after the Ending Credits which is extra
    script = file_clean.split('[End Credits]')[0]

    #removes the data before Act I, which is also extra
    #unfortunetly, website uses 2 different conventions, so I split depending on which one it uses
    
    if "Act I</b></u>" in script:
        script_clean = script.split("Act I</b></u>")[1]
        #bold_split = script_clean.split('<b>')
    elif "Act I</u></b>" in script:
        script_clean = script.split("Act I</u></b>")[1]
        #bold_split = script_clean.split('<b>')
    #return(script_clean)
    bold_split = script_clean.split('<b>')
    
    #creates a list of lists with both character + relevant dialogue
    master_list = []
    for x in range(len(bold_split)):
        output = bold_split[x].split(':')
        master_list.append(output)
    
    #collects all the characters who speak
    character_list = []

    for x in range(len(master_list)):
        holder = master_list[x]
        if len(holder)>1:
            test = holder[0].strip()
            character_list.append(test.strip('</b>')) # remove the bold hyperlink to cleanup the data
    #collects what all the characters say
    dialogue_list = []

    for x in range(len(master_list)):
        holder = master_list[x]
        if len(holder)>1:
            test = holder[0].strip()
            dialogue_list.append(holder[1].strip('</b>')) # remove the bold hyperlink to cleanup the data 
    
    
    #create a list of episode numbers to add to df (feels inefficient and should be a better way to do this)
    episode_list = []
    
    for x in range(len(character_list)):
        episode_list.append(int(episode_code['x']))
    
    dfScript = pd.DataFrame()
    
    dfScript['episode_code'] = episode_list
    
    dfScript['character_list'] = character_list
    dfScript['dialogue_list'] = dialogue_list
    
    count = dfScript['dialogue_list'].str.split().str.len()
    count = list(count)
    dfScript['word_count']=count

    return(dfScript)

In [ ]:
test = season_episode_list[:4]

In [ ]:
# the fact that this works, suggests that somethign else is going on... 

for i in test:
    print(dfConsolidateScript(i))
    #print(i)

In [ ]:
blankList = []

In [ ]:
for i in test:
    holder = dfConsolidateScript(i)
    blankList.append(holder)

In [ ]:
blankList

In [ ]:
df_list_holder = []

for i in range(len(test)):
    print(test[i])
    script = dfConsolidateScript(i)
    df_list_holder.append(script)

In [5]:
def dfConsolidateScript(x):

    dfScript = scriptClean(x)
    
    i = 1

    dfScriptFinal = pd.DataFrame()
    dfScriptFinal = dfScriptFinal.append(dfScript.iloc[0]) #return another function

    while i < len(dfScript.index):
        if dfScript['character_list'][i] == dfScript['character_list'][i-1]: 

            #consolidates the word count into a single value when the same characters speaks twice in a row
            word_count_consolidate = (dfScript['word_count'][i]+dfScript['word_count'][i-1])
            dfScriptFinal['word_count'].at[i-1] = word_count_consolidate 

            #consolidates the dialogue when the same character speaks twice in a row
            dialogue_consolidate = (dfScript['dialogue_list'][i]+ " " + dfScript['dialogue_list'][i-1])
            dfScriptFinal['dialogue_list'].at[i-1] = dialogue_consolidate 

            i = i + 1
        else:
            dfScriptFinal = dfScriptFinal.append(dfScript.iloc[i])
            i = i + 1
    dfScriptFinal['episode_code'] = dfScriptFinal['episode_code'].astype(int)
    return(dfScriptFinal)

In [ ]:
scriptClean(104)

In [ ]:
dfConsolidateScript(104)

In [ ]:
test = season_episode_list[:4]

In [ ]:
test

In [6]:
#create master data frame that aggregates all episodes & dialogue together
#pulls from episode list created in the beginning

#This is now apparently working. The Len(Range) was giving me problems. But I'm not sure why.

#can take some time and clean out stuff I don't need 


df_list_holder = []

for i in season_episode_list:
    #print(test[i])
    script = dfConsolidateScript(i)
    df_list_holder.append(script)

df_master = pd.concat(df_list_holder)
df_master.reset_index(drop = True, inplace = True)

UnboundLocalError: local variable 'script_clean' referenced before assignment

In [7]:
df_master

NameError: name 'df_master' is not defined

In [ ]:
df_master = pd.concat(df_list_holder)
df_master.reset_index(drop = True, inplace = True)

In [ ]:
df_master

In [8]:
dfConsolidateScript(101)

,character_list,dialogue_list,episode_code,word_count
0,Sokka,It's not getting away from me this time. Watc...,101,19.0
1,Katara,"Sokka, look!",101,2.0
2,Sokka,"Shhh. Katara, you're gonna scare it away. Mm...",101,14.0
3,Katara,Hey! But Sokka! I caught one!,101,6.0
5,Sokka,Ugh! Why is it that every time you play with ...,101,15.0
...,...,...,...,...
158,Katara,"I'm sorry, Aang. Maybe somehow there's a brig...",101,12.0
159,Aang,I did get to meet you.,101,6.0
160,Katara,Aang? Let's head back. This place is creepy. ...,101,15.0
162,Aang,What's that you said about booby traps? Huh?,101,8.0


In [9]:
dfConsolidateScript(102)

,character_list,dialogue_list,episode_code,word_count
0,Children,Yay! Aang's back!,102,3.0
1,Sokka,I knew it! You signaled the Fire Navy with th...,102,19.0
2,Katara,Aang didn't do anything. It was an accident.,102,8.0
3,Aang,"Yeh, we were on the ship and there was this b...",102,19.0
4,Gran Gran,"Katara, you shouldn't have gone on that ship....",102,15.0
...,...,...,...,...
122,Sokka,I'd like that. I'd really like that.,102,7.0
123,Katara,Then we're in this together.,102,5.0
124,Aang,"All right, but before I learn waterbending, w...",102,19.0
125,Katara,What's there?,102,2.0


In [ ]:
#scriptClean1(101)
#scriptClean2(101)
#dfConsolidateScript(104)